In [3]:
%use adventOfCode

In [4]:
val puzzle = AocClient.fromEnv().interactiveDay(2024, 11)
puzzle.viewPartOne()

--- Day 11: Plutonian Pebbles --- 
 The ancient civilization on Pluto was known for its ability to manipulate spacetime, and while The Historians explore their infinite corridors, you've noticed a strange set of physics-defying stones. 
 At first glance, they seem like normal stones: they're arranged in a perfectly straight line , and each stone has a number engraved on it. 
 The strange part is that every time you blink , the stones change . 
 Sometimes, the number engraved on a stone changes. Other times, a stone might split in two , causing all the other stones to shift over a bit to make room in their perfectly straight line. 
 As you observe them for a while, you find that the stones have a consistent behavior. Every time you blink, the stones each simultaneously change according to the first applicable rule in this list: 
 
 If the stone is engraved with the number 0 , it is replaced by a stone engraved with the number 1 . 
 If the stone is engraved with a number that has an even number of digits, it is replaced by two stones . The left half of the digits are engraved on the new left stone, and the right half of the digits are engraved on the new right stone. (The new numbers don't keep extra leading zeroes: 1000 would become stones 10 and 0 .) 
 If none of the other rules apply, the stone is replaced by a new stone; the old stone's number multiplied by 2024 is engraved on the new stone. 
 
 No matter how the stones change, their order is preserved , and they stay on their perfectly straight line. 
 How will the stones evolve if you keep blinking at them? You take a note of the number engraved on each stone in the line (your puzzle input). 
 If you have an arrangement of five stones engraved with the numbers 0 1 10 99 999 and you blink once, the stones transform as follows: 
 
 The first stone, 0 , becomes a stone marked 1 . 
 The second stone, 1 , is multiplied by 2024 to become 2024 . 
 The third stone, 10 , is split into a stone marked 1 followed by a stone marked 0 . 
 The fourth stone, 99 , is split into two stones marked 9 . 
 The fifth stone, 999 , is replaced by a stone marked 2021976 . 
 
 So, after blinking once, your five stones would become an arrangement of seven stones engraved with the numbers 1 2024 1 0 9 9 2021976 . 
 Here is a longer example: 
 Initial arrangement:
125 17

After 1 blink:
253000 1 7

After 2 blinks:
253 0 2024 14168

After 3 blinks:
512072 1 20 24 28676032

After 4 blinks:
512 72 2024 2 0 2 4 2867 6032

After 5 blinks:
1036288 7 2 20 24 4048 1 4048 8096 28 67 60 32

After 6 blinks:
2097446912 14168 4048 2 0 2 4 40 48 2024 40 48 80 96 2 8 6 7 6 0 3 2
 
 In this example, after blinking six times, you would have 22 stones. After blinking 25 times, you would have 55312 stones! 
 Consider the arrangement of stones in front of you. How many stones will you have after blinking 25 times?

In [5]:
import utils.longsOrZero

val testInput = "125 17"

class Solution {
    val knownLengths: MutableMap<Pair<Long, Int>, Long> = mutableMapOf()
    fun parse(input: String) = input.longsOrZero()

    fun blink(stone: Long): List<Long> {
        if (stone == 0L)
            return listOf(1)
        val stoneText = stone.toString()
        val length = stoneText.length
        if (length % 2 == 0)
            return listOf(stoneText.substring(0, length / 2).toLong(), stoneText.substring(length / 2, length).toLong())
        return listOf(stone * 2024L)
    }

    fun blink(stone: Long, times: Int): Long {
        val key: Pair<Long, Int> = stone to times
        return knownLengths.getOrPut(key) {
            if (times == 0) 1L else blink(stone).sumOf { blink(it, times - 1) }
        }
    }

    fun blink(stones: List<Long>, times: Int): Long = stones.sumOf { blink(it, times) }

    fun part1(input: String): Long = blink(parse(input), 25)
    fun part2(input: String): Long = blink(parse(input), 75)
}
Solution().part2(testInput)


65601038650482

In [6]:
Solution().part1(puzzle.input())


217812

In [7]:
//parse(testInput)
Solution().part2(puzzle.input())


259112729857522

In [10]:
object AltSolution {
    fun blink(stone: Long): List<Long> = when (stone) {
        0L -> listOf(1L)
        in (10..99) -> listOf(stone / 10L, stone % 10L)
        in (10_00..99_99) -> listOf(stone / 100L, stone % 100L)
        in (100_000..999_999) -> listOf(stone / 1000L, stone % 1000L)
        in (1000_0000..9999_9999) -> listOf(stone / 10000L, stone % 10000L)
        in (10000_00000..99999_99999) -> listOf(stone / 100000L, stone % 100000L)
        in (100000_000000..999999_999999) -> listOf(stone / 1000000L, stone % 1000000L)
        else -> listOf(stone * 2024L)
    }

    fun blink(stones: Map<Long, Long>): Map<Long, Long> =
        stones.entries.flatMap { (stone, count) -> blink(stone).map { it to count } }
            .groupingBy { it.first }.fold(0L) { acc, (_, count) -> acc + count }

    fun solve(input: String, times: Int): Map<Long, Long> {
        val stones = input.longsOrZero().groupingBy { it }.eachCount().mapValues { it.value.toLong() }
        return iterate(stones, times)
    }

    fun iterate(stones: Map<Long, Long>, times: Int): Map<Long, Long> {
        var result = stones
        repeat(times) {
            result = blink(result)
        }
        return result
    }
}
//val s = AltSolution.solve(puzzle.input(), 75)
// s.entries.sortedBy { it.key }.joinToString("\n")
// s.entries.sortedBy { it.key.toBigInteger() * it.value.toBigInteger() }.joinToString("\n")
//s.entries.size
val r1 = (0..<1_000L).associateWith { 1L }
val r2 = (0..<100_000L).associateWith { 1L }
val s1 = AltSolution.iterate(r1, 999)
val s2 = AltSolution.iterate(r2, 999)
"""${s1.size}, ${s2.size}, ${s2.keys - s1.keys}"""
val set1 =(0..1000).runningFold(r1) { acc, i -> AltSolution.blink(acc) }.takeLast(10).flatMapTo(mutableSetOf()) { it.keys }
val set2 =(0..1000).runningFold(r2) { acc, i -> AltSolution.blink(acc) }.takeLast(10).flatMapTo(mutableSetOf()) { it.keys }
(0..1000).runningFold(r1) { acc, i -> AltSolution.blink(acc) }.takeLast(10).joinToString("\n") {it.size.toString()}
listOf(set1.size, set2.size, set2-set1)

[3811, 3826, [60195784, 28692224, 1469424, 327726080000, 6019, 5784, 2869, 2224, 2974114176, 327726, 80000, 29741, 14176, 726, 161920000]]